In [1]:
#importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score
sns.set()

In [14]:
#reading in necessary csv files
users=pd.read_csv('takehome_users.csv',encoding='latin-1')
engagement=pd.read_csv('takehome_user_engagement.csv',encoding='latin-1')

In [15]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [45]:
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [62]:
#changing time_stamp to datetime object for ease of use
engagement['time_stamp']=pd.to_datetime(engagement['time_stamp'],infer_datetime_format=True)
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [63]:
print(type(engagement['time_stamp'][0]))

<class 'pandas._libs.tslib.Timestamp'>


In [91]:
#reformatting table
temp=pd.pivot_table(engagement, values='visited', index=['user_id', 'time_stamp'])

In [138]:
temp.head()

visited
user_id time_stamp                  
1       2014-04-22 03:53:30        1
2       2013-11-15 03:45:04        1
        2013-11-29 03:45:04        1
        2013-12-09 03:45:04        1
        2013-12-25 03:45:04        1

In [128]:
#getting largest number of logins in per week for each user through reformating
#resampling of multiindex for every 2 weeks
level_values = temp.index.get_level_values
temp_2 = (temp.groupby([level_values(i) for i in [0,1]] +[pd.Grouper(freq='1w', level=-1)])).count()

#formatting
temp_2=temp_2.reset_index(level=1)
temp_2=temp_2.drop(['time_stamp'],axis=1)
temp_2=temp_2.reset_index()

#grouping by user_id and timestamp and formatting to take max
temp_3=temp_2.groupby(['user_id','time_stamp']).sum()
temp_3=temp_3.reset_index()
temp_3=temp_3.groupby('user_id').max()
final=temp_3.drop('time_stamp',axis=1)
final=final.reset_index()

In [139]:
final.head()

,user_id,visited
0,1,1
1,2,3
2,3,1
3,4,1
4,5,1


In [218]:
#merging two datasets
total_df=users.merge(final, how='left',  left_on='object_id', right_on='user_id')
total_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,3.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,3.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,4.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,5.0,1.0


In [219]:
total_df=total_df.drop(['user_id'],axis=1)

In [220]:
total_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,3.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1.0


In [221]:
total_df['adopted']=total_df.visited>=3

In [222]:
total_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,visited,adopted
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,3.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1.0,False
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1.0,False
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1.0,False


Now, I have the data frame that I want, with the adopted true or false column for predictor purpose I can be begin to investigate factors that contribute to adoption.  I will also drop columns, such as name and email that have obviously no connection to adoption.

In [223]:
total_df=total_df.drop(['object_id','creation_time','name','email','last_session_creation_time','visited','invited_by_user_id'],axis=1)
total_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
0,GUEST_INVITE,1,0,11,False
1,ORG_INVITE,0,0,1,True
2,ORG_INVITE,0,0,94,False
3,GUEST_INVITE,0,0,1,False
4,GUEST_INVITE,0,0,193,False


In [224]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 5 columns):
creation_source               12000 non-null object
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
adopted                       12000 non-null bool
dtypes: bool(1), int64(3), object(1)
memory usage: 480.5+ KB


In [225]:
total_df=total_df.dropna()

In [226]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 5 columns):
creation_source               12000 non-null object
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
adopted                       12000 non-null bool
dtypes: bool(1), int64(3), object(1)
memory usage: 480.5+ KB


In [242]:
y=total_df['adopted'].values
df_x=total_df.drop(['adopted'],axis=1)
df_x=pd.get_dummies(df_x,drop_first=False)
X=df_x.values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [237]:
logreg=LogisticRegression()
param_grid = {'C':[0.01,0.1,1,10,100]}
logreg_cv = GridSearchCV(logreg,param_grid,cv=5)

logreg_cv.fit(X_train,y_train)
predicted=logreg_cv.predict(X_test)
score=logreg_cv.score(X_train,y_train)
print(score)

0.879523809524


In [244]:
logreg.fit(X_train,y_train)
coef=logreg.coef_
columns=df_x.columns

df_coef=pd.DataFrame(data=coef[0],index=columns)
df_ranked=df_coef.sort_values(0)
df_ranked

,0
creation_source_PERSONAL_PROJECTS,-0.900256
creation_source_ORG_INVITE,-0.398354
creation_source_SIGNUP,-0.256974
creation_source_GUEST_INVITE,-0.146968
creation_source_SIGNUP_GOOGLE_AUTH,-0.146739
org_id,0.001390
opted_in_to_mailing_list,0.040606
enabled_for_marketing_drip,0.060018


Here we see that the Creation Souce is the strongest factor for predicting future user adoption.  With Org_id, opted_in_to mailing_list, and enabled_for_marketing_drip having little effect on user adoption.  I will now create some graphs to back up this information

In [245]:
total_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
0,GUEST_INVITE,1,0,11,False
1,ORG_INVITE,0,0,1,True
2,ORG_INVITE,0,0,94,False
3,GUEST_INVITE,0,0,1,False
4,GUEST_INVITE,0,0,193,False


In [249]:
creation=total_df.groupby('creation_source').mean()
creation

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
creation_source,,,,
GUEST_INVITE,0.250116,0.147480,141.343504,0.150717
ORG_INVITE,0.253409,0.151387,138.940762,0.115421
PERSONAL_PROJECTS,0.242539,0.143534,144.294173,0.069635
SIGNUP,0.251557,0.152851,146.248203,0.131768
SIGNUP_GOOGLE_AUTH,0.244043,0.149458,141.523466,0.148736


In [250]:
opted=total_df.groupby('opted_in_to_mailing_list').mean()
opted

,enabled_for_marketing_drip,org_id,adopted
opted_in_to_mailing_list,,,
0,0.049967,141.639129,0.118477
1,0.448230,142.622912,0.126253


In [252]:
enabled=total_df.groupby('enabled_for_marketing_drip').mean()
enabled

,opted_in_to_mailing_list,org_id,adopted
enabled_for_marketing_drip,,,
0,0.161834,141.402527,0.119612
1,0.748884,144.630580,0.125000


These above statistics confirm our observation from the coef matrix of the logistic regression. We can see that the creation_source has the strongest impact on user adoption. Also notice that the avg value for the other columns other than adopted dont seem to vary very much indicated the two arent strongly correlated.  


Therefore, again, creation source of the account is the best predictor of user adoption with guest_invite being the best positive predictor.